In [1]:
import random
import textwrap
import datetime as dt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

In [2]:
df = pd.read_csv('globalterrorismdb_0718dist.csv', low_memory=False, usecols=['iyear', 'imonth', 'iday', 'country_txt', 'city', 
                                                                              'longitude', 'latitude', 'nkill', 'nwound', 
                                                                              'summary', 'target1', 'gname', 'attacktype1_txt', 'region_txt'])
df.rename(columns={'iyear':'year', 'imonth':'month', 'iday':'day', 'country_txt':'country', 'city':'city', 'longitude':'longitude', 
                   'latitude':'latitude', 'nkill':'killed', 'nwound':'wounded', 'summary':'summary', 'target1':'target', 
                   'gname':'group_name', 'attacktype1_txt':'attack_type', 'region_txt':'region'}, inplace=True)

In [3]:
# removing rows with month=0
df = df[df['month']!=0]

In [4]:
# removing rows with month=0
df = df[df['day']!=0]

In [5]:
df['date'] = [dt.datetime(y,m,d) for y, m, d in zip(df['year'], df['month'], df['day'])]

In [6]:
df.head()

,year,month,day,country,region,city,latitude,longitude,summary,attack_type,target,group_name,killed,wounded,date
0,1970,7,2,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,NaN,Assassination,Julio Guzman,MANO-D,1.0,0.0,1970-07-02
5,1970,1,1,United States,North America,Cairo,37.005105,-89.176269,1/1/1970: Unknown African American assailants ...,Armed Assault,Cairo Police Headquarters,Black Nationalists,0.0,0.0,1970-01-01
6,1970,1,2,Uruguay,South America,Montevideo,-34.891151,-56.187214,NaN,Assassination,Juan Maria de Lucah/Chief of Directorate of in...,Tupamaros (Uruguay),0.0,0.0,1970-01-02
7,1970,1,2,United States,North America,Oakland,37.791927,-122.225906,1/2/1970: Unknown perpetrators detonated explo...,Bombing/Explosion,Edes Substation,Unknown,0.0,0.0,1970-01-02
8,1970,1,2,United States,North America,Madison,43.076592,-89.412488,"1/2/1970: Karl Armstrong, a member of the New ...",Facility/Infrastructure Attack,"R.O.T.C. offices at University of Wisconsin, M...",New Year's Gang,0.0,0.0,1970-01-02


In [7]:
app = dash.Dash()
server = app.server
app.title = 'Global Terrorist Attack Data Visualization Dashboard'

In [8]:
# app layout
app.layout = html.Div([
    dcc.Graph(id='map', config={'displayModeBar': False}),

    html.Div([
        dcc.RangeSlider(id='years_1',
                        min=df.year.min(),
                        max=df.year.max(),
                        dots=True,
                        value=[2010, df.year.max()],
                        marks={str(yr): "'" + str(yr)[2:] for yr in range(df.year.min(), df.year.max()+1)}),
         
        html.Br(),
        html.Br()
    ], style={'width':'80%', 'margin-left':'10%', 'background-color':'#f2f2f2'}),
    html.Div([
        dcc.Dropdown(id='countries',
                     multi=True,
                     value=[''],
                     placeholder='Select Countries from here',
                     options=[{'label':i, 'value':i} for i in sorted(df['country'].unique())]),
        html.Br(),
        html.Br()
    ], style={'width':'60%', 'margin-left':'20%', 'background-color':'#f2f2f2'}),
    
    dcc.Graph(id='country_by_year', config={'displayModeBar':False}),
    html.Hr(), 
#     html.Content('Top Countries', style={'font-family':'Palatino', 'margin-left':'45%', 'font-size':25}),
    html.Br(),

    html.Div([
        html.Div([
            html.Div([
                dcc.RangeSlider(id='years_2',
                                min=df.year.min(),
                                max=df.year.max(),
                                dots=True,
                                value=[2012, df.year.max()],
                                marks={str(y): str(y) for y in range(df.year.min(), df.year.max()+1, 5)}),
                html.Br(),                
            ], style={'margin-left':'8%', 'margin-right':'1%'}),
            dcc.Graph(id='country_attacks', config={'displayModeBar':False}, figure={'layout':{'margin':{'r':10, 't':50}}}),
            html.Hr(),
            html.Br(),
            dcc.Graph(id='deaths', config={'displayModeBar':False}, figure={'layout':{'margin':{'r':10, 't':50}}})
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            html.Div([
                dcc.RangeSlider(id='years_3',
                                min=df.year.min(),
                                max=df.year.max(),
                                dots=True,
                                value=[2010, df.year.max()],
                                marks={str(yr): str(yr) for yr in range(df.year.min(), df.year.max()+1, 5)}),
                html.Br(),
                
            ], style={'margin-left':'1%', 'margin-right':'3%'}),

            dcc.Graph(id='country_deaths', config={'displayModeBar':False}, figure={'layout':{'margin':{'l':10, 't':50}}}),
            html.Hr(),
            html.Br(),
            dcc.Graph(id='g_name', config={'displayModeBar':False}, figure={'layout':{'margin':{'l':10, 't':50}}}),
        ], style={'width':'48%', 'display':'inline-block', 'float':'right'}),
    ]),
    
    html.P(),
    html.Content('Global Terrorism Data from 1970 till 2017')    
], style={'background-color': '#e9e2e2e'})

In [9]:

# first callback
@app.callback(Output('country_by_year', 'figure'),
             [Input('countries', 'value'), Input('years_1', 'value')])
def countryBarchatYear(countries, years):
    df_cpy = df[df['country'].isin(countries) & df['year'].between(years[0], years[1])]
    df_cpy = df_cpy.groupby(['year', 'country'], as_index=False)['date'].count()
    
    return {
        'data': [go.Bar(x=df_cpy[df_cpy['country'] == i]['year'],
                        y=df_cpy[df_cpy['country'] == i]['date'], name=i) for i in countries] ,
        'layout': go.Layout(title='Yearly Terrorist Attacks ' + ', '.join(countries) + '  ' + ' - '.join([str(y) for y in years]),
                            plot_bgcolor='#f2f2f2',
                            paper_bgcolor='#f2f2f2',
                            font={'family': 'Baskerville'},
                            titlefont={'size': 24})
    }

In [10]:
# second callback
@app.callback(Output('map', 'figure'),
        [Input('countries', 'value'), Input('years_1', 'value')])


def CountryOnMap(countries, years):
    df_com = df[df['country'].isin(countries) & df['year'].between(years[0], years[1])]
    
    return {
        'data': [go.Scattergeo(lon=[x + random.gauss(0.04, 0.03) for x in df_com[df_com['country'] == i]['longitude']],
                               lat=[x + random.gauss(0.04, 0.03) for x in df_com[df_com['country'] == i]['latitude']],
                               name=i,
                               hoverinfo='text',
                               opacity=0.9,
                               marker={'size':9, 'line':{'width':0.2, 'color':'#cccccc'}},
                               hovertext= df_com[df_com.country==i].city.astype(str) + ', ' + df_com[df_com.country==i].country.astype(str) + '<br>' + 
                                            [dt.datetime.strftime(d, '%d %B %Y') for d in df_com[df_com.country==i].date] + '<br>' + 
                                            'Perpetrator: ' + df_com[df_com.country==i].group_name.astype(str) + '<br>' + 
                                            'Killed: ' + df_com[df_com.country==i].killed.astype(str) + '<br>' + 
                                            'Wounded: ' + df_com[df_com.country==i].wounded.astype(str) + '<br>' + 
                                            'Target: ' + df_com[df_com.country==i].target.astype(str) + '<br>' + 
                                            ['<br>'.join(textwrap.wrap(x, 40)) if not isinstance(x, float) else "" for x in df_com[df_com.country==i].summary])
                        for i in countries],
        
        'layout': go.Layout(title='Terrorist Attacks ' + ', '.join(countries) + '  ' + ' - '.join([str(y) for y in years]),
                            font={'family': 'Baskerville'},
                            titlefont={'size': 24},
                            paper_bgcolor='#f2f2f2',
                            plot_bgcolor='#f2f2f2',
                            width=1420,
                            height=650,
                            geo={'showland': True, 'landcolor': '#f2f2f2',
                                 'countrycolor': '#bebebe',
                                 'showsubunits': True,
                                 'subunitcolor': '#bebebe',
                                 'subunitwidth': 5,
                                 'showcountries': True,
                                 'oceancolor': '#a8e6ff',
                                 'showocean': True,
                                 'showcoastlines': True, 
                                 'showframe': False,
                                 'coastlinecolor': '#006994',
                                 'lonaxis': {'range': [df_com.longitude.min()-1, df_com.longitude.max()+1]},
                                 'lataxis': {'range': [df_com.latitude.min()-1, df_com.latitude.max()+1]}
                                              })
    }

In [11]:
#third callback
@app.callback(Output('country_attacks', 'figure'),
        [Input('countries', 'value') ,Input('years_2', 'value')])

def active_terrorist_group(countries, years):
    df_atg = df[df['country'].isin(countries) & df['year'].between(years[0], years[1])]
    df_atg = df_atg.groupby(['country', 'group_name'], as_index=False)['date'].count().sort_values('date', ascending=False)#[:20]
    
    return {
        'data': [go.Bar(x=df_atg[df_atg.country == i].group_name[:5],
                        y=df_atg[df_atg.country == i].date[:5], name=i) for i in countries],
        'layout': go.Layout(title='Top 5 Active Terrorist Group in ' + ', '.join(countries) + '  ' + ' - '.join([str(y) for y in years]),
                            plot_bgcolor='#f2f2f2',
                            paper_bgcolor='#f2f2f2',
                            font={'family': 'Baskerville'},
                            titlefont={'size': 24})
    }

In [12]:
# fourth callback
@app.callback(Output('country_deaths', 'figure'),
        [Input('countries', 'value'), Input('years_3', 'value')])

def active_group_target_places(countries, years):
    df_tp = df[df['country'].isin(countries) & df['year'].between(years[0], years[1])]
    df_tp = df_tp.groupby(['country', 'target'], as_index=False)['date'].count().sort_values('date', ascending=False)
    
    return {
        'data': [go.Bar(x=df_tp[df_tp.country == i].target[:5],
                        y=df_tp[df_tp.country == i].date[:5], 
                        name=i) for i in countries],
        'layout': go.Layout(title='Top 5 Targets by Terrorist Groups in ' + ', '.join(countries) + '  ' + ' - '.join([str(y) for y in years]),
                            plot_bgcolor='#f2f2f2',
                            paper_bgcolor='#f2f2f2',
                            font={'family': 'Baskerville'},
                            titlefont={'size': 24})
    }

In [13]:
# fifth callback
@app.callback(Output('g_name', 'figure'),
        Input('years_3', 'value'))

def countries_attacked(years):
    df_gn = df[df.year.between(years[0], years[1])]
    df_gn = df_gn.groupby('group_name', as_index=False).date.count().sort_values('date', ascending=False)
    
    return {
        'data': [go.Bar(x=df_gn['date'][:15],
                        y=df_gn.group_name[:15],
                        showlegend=False,
                        constraintext='none',
                        orientation='h',
                        text=df_gn.group_name[:15],
                        textposition='outside')],
        'layout': go.Layout(title='Most Active Terrorist Groups during' + ' ' + ' - '.join([str(y) for y in years]),
                            plot_bgcolor='#f2f2f2',
                            paper_bgcolor='#f2f2f2',
                            font={'family': 'Baskerville'},
                            titlefont={'size': 24},
                            yaxis={'visible': False})
    }

In [14]:
# sixth callback
@app.callback(Output('deaths', 'figure'),
        Input('years_2', 'value'))

def countries_attacked(years):
    df_wd = df[df.year.between(years[0], years[1])]
    df_wd = df_wd.groupby('country', as_index=False).killed.agg(['sum', 'count']).sort_values('sum', ascending=False)
    
    return {
        'data': [go.Bar(x=df_wd['sum'][:15],
                        y=df_wd.index[:15],
                        showlegend=False,
                        constraintext='none',
                        orientation='h',
                        text=df_wd.index[:15],
                        textposition='outside')],
        'layout': go.Layout(title='Top 15 Countries by Deaths during' + ' ' + ' - '.join([str(y) for y in years]),
                            plot_bgcolor='#f2f2f2',
                            paper_bgcolor='#f2f2f2',
                            font={'family': 'Baskerville'},
                            titlefont={'size': 24},
                            yaxis={'visible': False})
    }

In [ ]:
if __name__ == '__main__':
	app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jun/2021 23:38:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:35] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:35] "GET /_dash-layout HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 2051, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1501, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1499, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1485, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\dash\d

127.0.0.1 - - [18/Jun/2021 23:38:35] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 2051, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1501, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1499, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1485, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\dash\d

...

127.0.0.1 - - [18/Jun/2021 23:38:35] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 2051, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1501, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1499, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1485, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\dash\d

127.0.0.1 - - [18/Jun/2021 23:38:35] "POST /_dash-update-component HTTP/1.1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 2051, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1501, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1499, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\flask\app.py", line 1485, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Arvind documents\comedians-project1\venvml\lib\site-packages\dash\d

" 500 -
127.0.0.1 - - [18/Jun/2021 23:38:35] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [18/Jun/2021 23:38:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:38:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:39:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23:39:03] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 23: